In [3]:
# datasets are mixture of numbers and strings. So we cannot use a label encoder.
# using Naive Bayes Classifier
import numpy as np
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split,cross_val_score

# load datasets
input_file = 'adult.data.txt'
X,Y = [],[]
count_lessthan50k ,count_morethan50k = 0, 0
num_images_threshold = 10000

with open(input_file,'r') as f:
    for line in f.readlines():
        if '?' in line:
            continue
        data = line[:-1].split(',')
        if data[-1] == '<=50K' or count_lessthan50k < num_images_threshold:
            X.append(data)
            count_lessthan50k += 1
        elif data[-1] == '>50K' and count_morethan50k < num_images_threshold:
            X.append(data)
            count_morethan50k += 1
        if count_lessthan50k >= num_images_threshold and count_morethan50k >= num_images_threshold:
            break

X = np.array(X)

# convert string data to numerical data
label_encoder = []
X_encoded = np.empty(X.shape)
for i,item in enumerate(X[0]):
    if item.isdigit():
        X_encoded[:,i] = X[:,i]
    else:
        label_encoder.append(preprocessing.LabelEncoder())
        X_encoded[:,i] = label_encoder[-1].fit_transform(X[:,i])

X = X_encoded[:,:-1].astype(int)
Y = X_encoded[:,-1].astype(int)

# train the classifier
classifier_gaussiannb = GaussianNB()
classifier_gaussiannb.fit(X,Y)

# cross validation
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, random_state = 5)
classifier_gaussiannb = GaussianNB()
classifier_gaussiannb.fit(X_train, Y_train)
Y_test_pred = classifier_gaussiannb.predict(X_test)

# extract performance metrics
f1 = cross_val_score(classifier_gaussiannb, X, Y, scoring="f1_weighted", cv=5)
print("\nF1 Score:",round(100*f1.mean(),2),"%")




F1 Score: 78.42 %


In [22]:
''' 没搞定 label_encoder
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(input_data)
for i,item in enumerate(input_data):
    print(i, list(label_encoder.transform(input_data))[i])
print(input_data_encoded)
'''

# how to classify a single datapoint
input_data = ['39', 'State-gov', '77516', 'Bachelors', '13','Never-married', 'Adm-clerical', 'Not-in-family', 'White', 'Male','2174', '0', '40', 'United-States']
count = 0
input_data_encoded = [-1] * len(input_data)
for i,item in enumerate(input_data):
    if item.isdigit():
        input_data_encoded[i] = int(input_data[i])
    else:
        label_encoder.append(preprocessing.LabelEncoder())
        input_data_encoded[i] = int(label_encoder[count].fit(X[:,i]))
        #input_data_encoded[i] = label_encoder[count].transform(np.array(input_data))
        count += 1
input_data_encoded = np.array(input_data_encoded)

output_class = classifier_gaussiannb.predict(input_data_encoded)
print(label_encoder[-1].inverse_transform(output_class)[0])

TypeError: int() argument must be a string, a bytes-like object or a number, not 'LabelEncoder'